# Creación MasterTable

In [250]:
import numpy as np
import astropy.units as u
from astropy.table import Table, QTable, join,vstack
from astropy.coordinates import SkyCoord
import matplotlib.pyplot as plt


from astroquery.simbad import Simbad
from astropy.coordinates import Angle

from  astroquery.gaia import Gaia
import pandas as pd

1- Apertura tablas cruzadas

In [251]:
laura = Table.read('CatalogosRadio/laura.csv') #Leer tabla
laura=laura.to_pandas()
laura=laura.rename(columns={'Objeto':'Name'})      

'''
laura['Redshift']=laura['Redshift'].apply(lambda x: x.replace(',','.'))
laura['Magnitud(V)']=laura['Magnitud(V)'].apply(lambda x: x.replace(',','.'))

laura['Redshift']=laura['Redshift'].astype(float)
laura['Magnitud(V)']=laura['Magnitud(V)'].astype(float)

laura=Table.from_pandas(laura)
laura.write('../CatalogosRadio/laura.csv', overwrite=True)

'''
laura=Table.from_pandas(laura)

In [252]:
CM_L_S = Table.read('CatalogosRadio/CM_L_S',format='ascii.ecsv') #Leer tabla
CM_L_B= Table.read('CatalogosRadio/CM_L_B', format='ascii.ecsv') #Leer tabla
CM_L_B3 = Table.read('CatalogosRadio/CM_L_B3', format='ascii.ecsv') #Leer 
CM_L_N= Table.read('CatalogosRadio/CM_L_N', format='ascii.ecsv') #Leer tabla

2-MasterTable a partir de la función join

In [253]:
def join_two_tables (tab1,tab2,keys,join_type):
    joined = join(tab1,tab2,keys,join_type)
    
    return joined

Se crea una "MasterTable" de izquierda a derecha que contenga todos los LINERS e incluya la información que hay en los diferentes catálogos para ellos.

In [254]:
MergerLS= join_two_tables(laura, CM_L_S['Name','Beam','BPA','Speak','RMS'], keys="Name", join_type='left')
MergerLSB = join_two_tables(MergerLS, CM_L_B['Name','Beam','Q', 'PA'], keys='Name', join_type='left')
MergerLSBB3 = join_two_tables(MergerLSB, CM_L_B3['Name', 'det', 'morph','log(Lcore)', 'log(Ltotal)','logMBH','log[OIII]','logEddRatio'], keys='Name', join_type='left')
Finaltable = join_two_tables(MergerLSBB3, CM_L_N['Name','AType','Lp15GHz','Dist','St15GHz'], keys='Name', join_type='left')

type(Finaltable)


astropy.table.table.Table

## A partir de la MasterTable, creamos nuevas columnas para los upperlimits. Que son log(Lcore), log[OIII] y logEddRatio

In [255]:
#Pasamos la tabla a DataFrame para trabajar mejor.
Finaltable= Finaltable.to_pandas()
type(Finaltable)

pandas.core.frame.DataFrame

La función add_upperlimits nos devuelve la misma "FinalTable" pero en caso de que una columna (check_column) tenga valores con upperlimit (i.e <4.2), creará una nueva columna con dichos valores (sin signo) y los eliminará de la columna original.  
De esta forma la columna original (ej. LogEddRatio) solo contendrá floats y la nueva (ej. Upp_Lim_logEddRatio) contendrá los upperlimits también en type float. 

In [256]:
#Add upper limits
#Inicial_table: FinalTable
#Check_column : Columna a chequear 
#New_colum : Nombre nueva columna
#Posicion: Posicion de la nueva columna, numero. 

def add_upperlimits (Inicial_table, condition, check_column, new_column, posicion): 
    #Creamos una tabla. Contiene filas cuyo valor en la check_column se un upper limit (i.e el primer valor de la string sea uno de los descritos en la condición).
    tabla_1 = Inicial_table[Inicial_table[check_column].astype(str).str[0].isin(condition)]
    #Insertamos la nueva columna en la posicion que queramos, obviamos el primer valor del string(<,>) y lo pasamos a float.
    tabla_1.insert(posicion, new_column, tabla_1[check_column].astype(str).str[1:].astype(float))
    #Los valores en la check column pasan a ser '' en esta tabla.
    tabla_1[check_column] = ''
    #Creamos una tabla con las filas que NO cumplan esta condicion --> las que no tengan upperlimit.
    tabla_2 = Inicial_table[~Inicial_table[check_column].astype(str).str[0].isin(lista1)]
    
    #Juntamos ambas tablas
    tabla_union = pd.concat([tabla_1, tabla_2])

    tabla_union.sort_index(inplace=True)   
    

    return tabla_union


# Importante:  

1. Cuando llamemos a la función es importante que la "Initial_table" sea un DataFrame.  
2. Las siguientes celdas solo se pueden correr 1 vez. En caso contrario nos devuelve error ya que "La nueva columna ya está creada anteriormente". Hay que volver a crear la FinalTable, en las celdas anteriores.
3. Atención con la posición: al añadir una columna hay que tenerla en cuenta.  

A continuación se añaden las tres columnas de upperlimits: 

In [257]:
Finaltable = add_upperlimits(Finaltable, ['>', '<'], 'logEddRatio', 'Upp_Lim_logEddRatio', 19)


<ipython-input-256-e4ba84a09db3>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tabla_1[check_column] = ''


In [258]:
Finaltable = add_upperlimits(Finaltable,['>', '<'], 'log(Lcore)', 'Upp_Lim_log(Lcore)', 15)

<ipython-input-256-e4ba84a09db3>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tabla_1[check_column] = ''


In [259]:
Finaltable = add_upperlimits(Finaltable, ['>', '<'], 'log[OIII]', 'Upp_Lim_log[OIII]', 19)

<ipython-input-256-e4ba84a09db3>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tabla_1[check_column] = ''


In [260]:
Finaltable

,Name,RA,DEC,Redshift,Magnitud(V),Beam_1,BPA,Speak,RMS,Beam_2,...,log(Ltotal),logMBH,log[OIII],Upp_Lim_log[OIII],logEddRatio,Upp_Lim_logEddRatio,AType,Lp15GHz,Dist,St15GHz
0,IC1459,22 57 10.60,−36 27 44.0,0.0055,10.54,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NGC1052,02 41 04.80,+08 15 20.8,0.0049,11.04,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NGC2639,08 43 38.08,+50 12 20.0,0.0111,11.76,NaN,NaN,NaN,NaN,0.20x0.19,...,38.50,7.94,39.60,NaN,-2.93,NaN,NaN,NaN,NaN,NaN
3,NGC2655,08 55 37.73,+78 13 23.1,0.0047,10.39,NaN,NaN,NaN,NaN,0.20x0.20,...,37.97,7.74,39.44,NaN,-2.89,NaN,NaN,NaN,NaN,NaN
4,NGC266,00 49 47.80,+32 16 39.8,0.0155,11.82,NaN,NaN,NaN,NaN,0.48x0.31,...,37.02,8.37,39.43,NaN,-3.53,NaN,L1.9,21.280001,62.400002,4.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,NGC7331,22 37 04.09,+34 24 56.3,0.0038,9.41,NaN,NaN,NaN,NaN,0.15x0.15,...,-,8.02^{*},38.30,NaN,-3.76,NaN,T2,19.430000,14.300000,1.1
71,NGC7743,23 44 21.14,+09 56 02.7,0.0057,12.43,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
72,NGC833,02 09 20.88,-10 07 59,0.0129,12.90,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
73,NGC835,02 09 24.69,-10 08 10.,0.0136,12.41,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Finalmente, pasamos la tabla otra vez a Table (astropy) y la guardamos en .csv. 

In [261]:
Finaltable= Table.from_pandas(Finaltable)


In [262]:
Finaltable

Name,RA,DEC,Redshift,Magnitud(V),Beam_1,BPA,Speak,RMS,Beam_2,Q,PA,det,morph,log(Lcore),Upp_Lim_log(Lcore),log(Ltotal),logMBH,log[OIII],Upp_Lim_log[OIII],logEddRatio,Upp_Lim_logEddRatio,AType,Lp15GHz,Dist,St15GHz
str7,str12,str12,float64,float64,str9,float64,str10,float64,str9,str2,float64,str5,str1,str5,float64,str5,str8,str5,float64,str5,float64,str9,float32,float32,float32
IC1459,22 57 10.60,−36 27 44.0,0.0055,10.54,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--
NGC1052,02 41 04.80,+08 15 20.8,0.0049,11.04,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--
NGC2639,08 43 38.08,+50 12 20.0,0.0111,11.76,--,--,--,--,0.20x0.19,++,68.0,I,C,37.61,--,38.50,7.94,39.60,--,-2.93,--,--,--,--,--
NGC2655,08 55 37.73,+78 13 23.1,0.0047,10.39,--,--,--,--,0.20x0.20,+,0.0,I,E,37.59,--,37.97,7.74,39.44,--,-2.89,--,--,--,--,--
NGC266,00 49 47.80,+32 16 39.8,0.0155,11.82,--,--,--,--,0.48x0.31,+,-40.0,I,A,36.94,--,37.02,8.37,39.43,--,-3.53,--,L1.9,21.28,62.4,4.1
NGC2681,08 53 32.73,+51 18 49.3,0.0023,10.86,0.17x0.14,-87.2,0.34+-0.01,9.6,0.19x0.16,++,51.8,I,C,35.51,--,35.99,7.07,38.37,--,-3.29,--,L1.9,19.48,13.3,1.4
NGC2685,08 55 34.71,+58 44 03.83,0.003,11.35,--,--,--,--,--,--,--,U,-,,34.82,-,6.59^{*},38.41,--,-2.73,--,S2/T2,19.45,16.2,0.9
NGC2787,09 19 18.56,+69 12 12.0,0.0023,11.25,--,--,--,--,--,--,--,I,A,36.28,--,36.41,7.61^{*},38.37,--,-3.79,--,L1.9,20.15,13.0,7.0
NGC2841,09 22 02.63,+50 58 35.5,0.0021,10.18,--,--,--,--,0.19x0.16,++,52.7,I,C,34.84,--,35.47,8.31,38.19,--,-4.63,--,--,--,--,--


Se guarda la tabla

In [263]:
Finaltable.write('CatalogosRadio/Master_table.csv', overwrite=True) 

La pasamos a pandas por si fuera mejor para tratar datos.

In [264]:
Finaltable.to_pandas()

,Name,RA,DEC,Redshift,Magnitud(V),Beam_1,BPA,Speak,RMS,Beam_2,...,log(Ltotal),logMBH,log[OIII],Upp_Lim_log[OIII],logEddRatio,Upp_Lim_logEddRatio,AType,Lp15GHz,Dist,St15GHz
0,IC1459,22 57 10.60,−36 27 44.0,0.0055,10.54,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NGC1052,02 41 04.80,+08 15 20.8,0.0049,11.04,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NGC2639,08 43 38.08,+50 12 20.0,0.0111,11.76,NaN,NaN,NaN,NaN,0.20x0.19,...,38.50,7.94,39.60,NaN,-2.93,NaN,NaN,NaN,NaN,NaN
3,NGC2655,08 55 37.73,+78 13 23.1,0.0047,10.39,NaN,NaN,NaN,NaN,0.20x0.20,...,37.97,7.74,39.44,NaN,-2.89,NaN,NaN,NaN,NaN,NaN
4,NGC266,00 49 47.80,+32 16 39.8,0.0155,11.82,NaN,NaN,NaN,NaN,0.48x0.31,...,37.02,8.37,39.43,NaN,-3.53,NaN,L1.9,21.280001,62.400002,4.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,NGC7331,22 37 04.09,+34 24 56.3,0.0038,9.41,NaN,NaN,NaN,NaN,0.15x0.15,...,-,8.02^{*},38.30,NaN,-3.76,NaN,T2,19.430000,14.300000,1.1
71,NGC7743,23 44 21.14,+09 56 02.7,0.0057,12.43,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
72,NGC833,02 09 20.88,-10 07 59,0.0129,12.90,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
73,NGC835,02 09 24.69,-10 08 10.,0.0136,12.41,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
